In [1]:
import dill
from FindFiles import find_files
from Settings import Settings
from CoRefHelper import EMPTY
from collections import defaultdict
from BrattEssay import ANAPHORA

DATASET = "CoralBleaching" # CoralBleaching | SkinCancer
PARTITION = "Training" # Training | Test

settings = Settings()
root_folder = settings.data_directory + DATASET + "/Thesis_Dataset/"
merged_predictions_folder = root_folder + "CoReference/"

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [2]:
essay_files = find_files(merged_predictions_folder)
if PARTITION == "Training":
    essay_files = [e for e in essay_files if "train" in e]
assert len(essay_files) == 1
with open(essay_files[0], "rb") as f:
    essays = dill.load(f)
len(essays)

902

In [36]:
import pathlib
# ensure paths exist

output_root_dir = "/Users/simon.hughes/data/berkely_ner/{dataset}/{partition}".format(dataset=DATASET, partition=PARTITION)
output_dir = output_root_dir + "/text" 
pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True) 
print(output_dir)

/Users/simon.hughes/data/berkely_ner/CoralBleaching/Training/text


In [37]:
import os

PART = "0"
for e in essays:
    name = e.name[:-4] + ".txt"
    first_line = "#begin document ({name}); part 000".format(name=name)
    lines = [first_line]
    for sent in e.sentences:
        for wd_ix, (wd, _) in enumerate(sent):
            lines.append("{name}\t{part}\t{wd_ix}\t{word}".format(name=name, part=PART, wd_ix=wd_ix, word=wd))
        # insert blank line as sentence separator
        lines.append("")
    lines.append("#end document")
    lines.append("")
    contents = "\n".join(lines)
    fname = os.path.join(output_dir, name.replace(".txt",".conll"))
    
    with open(fname, "w+") as fin:
        fin.write(contents)


In [41]:
# Call Berkeley System
from subprocess import call
call(["/Users/simon.hughes/data/berkely_ner/run-test_parameterized.sh", output_root_dir])

1